In [1]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import warnings
from tqdm import tqdm
import pickle
from typing import Dict, List, Tuple, Union, Optional
import matplotlib.gridspec as gridspec
from scipy import stats

# Configuración general (sin cambios)
plt.style.use('seaborn-v0_8-darkgrid')
warnings.filterwarnings('ignore')

# Crear directorios para resultados
os.makedirs('./artifacts/results', exist_ok=True)
os.makedirs('./artifacts/results/figures', exist_ok=True)
os.makedirs('./artifacts/results/data', exist_ok=True)

# Configurar logging
logging.basicConfig(
    filename='./artifacts/errors.txt',
    level=logging.ERROR,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

class AMAR:
    """
    Implementación mejorada de la estrategia AMAR (Adaptive Multi-factor Allocation with Reinforcement learning)
    que combina identificación de regímenes de mercado con técnicas de aprendizaje por refuerzo
    para optimizar la selección de factores y construcción de portafolios.
    """
    
    def __init__(self,
                 start_date: str = '2010-01-01',
                 end_date: str = None,
                 lookback_years: int = 3,
                 regime_update_frequency: str = 'M',
                 portfolio_rebalance_frequency: str = 'W-FRI',
                 n_regimes: int = 3,
                 market_index: str = 'SPY',
                 target_beta: float = 0.0,  # Cambiado a 0.0 para enfatizar neutralidad al mercado
                 beta_range: float = 0.1,
                 max_stock_weight: float = 0.05,  # Aumentado para permitir más concentración en mejores señales
                 max_sector_deviation: float = 0.05,
                 max_turnover: float = 0.2,  # Reducido para limitar costos de transacción
                 volatility_target: float = 0.08,  # Reducido para mejor control de riesgo
                 min_liquidity: float = 5e6,  # Aumentado a $5M para mejor liquidez
                 transaction_cost: float = 0.0015,  # Más realista (0.15%)
                 random_state: int = 42):
        """
        Inicializa la estrategia AMAR con parámetros optimizados.
        """
        self.start_date = start_date
        self.end_date = end_date if end_date else datetime.now().strftime('%Y-%m-%d')
        self.lookback_years = lookback_years
        self.regime_update_frequency = regime_update_frequency
        self.portfolio_rebalance_frequency = portfolio_rebalance_frequency
        self.n_regimes = n_regimes
        self.market_index = market_index
        self.target_beta = target_beta
        self.beta_range = beta_range
        self.max_stock_weight = max_stock_weight
        self.max_sector_deviation = max_sector_deviation
        self.max_turnover = max_turnover
        self.volatility_target = volatility_target
        self.min_liquidity = min_liquidity
        self.transaction_cost = transaction_cost
        self.random_state = random_state
        
        # Atributos internos (sin cambios)
        self.data = None
        self.sp500_stocks = None
        self.market_data = None
        self.regimes = None
        self.factor_performance = {}
        self.portfolio_weights = pd.DataFrame()
        self.portfolio_returns = pd.Series(dtype='float64')
        self.current_regime = None
        
        # MEJORA 1: Añadir factor de Reversal a corto plazo
        self.factor_categories = ['Value', 'Momentum', 'Quality', 'Volatility', 'Liquidity', 'Reversal']
        
        # Inicializar distribuciones de creencia para Thompson Sampling
        self.initialize_belief_distributions()
        
        # Métricas de rendimiento
        self.metrics = {}
        
        # MEJORA 2: Añadir descarga de VIX para mejor identificación de regímenes
        self.vix_data = None
        
        np.random.seed(random_state)
        
    def initialize_belief_distributions(self):
        """
        Inicializa las distribuciones de creencia para el algoritmo Thompson Sampling.
        Usado prior informativo más balanceado basado en literatura de factores.
        """
        # Para cada categoría y cada régimen, mantenemos alpha y beta para distribución Beta
        self.belief_distributions = {}
        
        # MEJORA 3: Prior informativo mejorado basado en literatura académica
        priors = {
            'Value': {'success': 5, 'failure': 5},     # Valor funciona bien en regímenes de recuperación
            'Momentum': {'success': 6, 'failure': 4},  # Momentum tiende a funcionar en tendencias claras
            'Quality': {'success': 7, 'failure': 3},   # Quality tiende a ser más estable en general
            'Volatility': {'success': 5, 'failure': 5}, # Neutral para empezar
            'Liquidity': {'success': 4, 'failure': 6},  # Liquidez es importante en regímenes de crisis
            'Reversal': {'success': 4, 'failure': 6}    # Reversal funciona mejor en mercados volátiles
        }
        
        for category in self.factor_categories:
            self.belief_distributions[category] = {}
            for regime in range(self.n_regimes):
                # Inicializamos con priors informativas
                self.belief_distributions[category][regime] = {
                    'alpha': priors[category]['success'],
                    'beta': priors[category]['failure']
                }
    
    def get_sp500_tickers(self) -> List[str]:
        """
        Obtiene los tickers actuales del S&P 500 desde Wikipedia.
        
        Returns:
            Lista de tickers del S&P 500
        """
        try:
            table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
            df = table[0]
            tickers = df['Symbol'].str.replace('.', '-').tolist()
            return tickers
        except Exception as e:
            logging.error(f"Error al obtener tickers del S&P 500: {str(e)}")
            # Fallback a lista reducida en caso de error
            return ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'BRK-B', 'JPM', 'JNJ', 'V', 'PG', 'UNH', 'HD', 'BAC', 'MA']
    
    def download_data(self):
        """
        Descarga datos históricos para todos los activos del S&P 500, índice de referencia y VIX.
        """
        try:
            # Calcular fecha de inicio para incluir el período de lookback
            extended_start = (datetime.strptime(self.start_date, '%Y-%m-%d') - 
                             timedelta(days=int(365.25 * self.lookback_years))).strftime('%Y-%m-%d')
            
            # Obtener tickers del S&P 500
            self.sp500_stocks = self.get_sp500_tickers()
            
            # MEJORA 4: Añadir VIX y otros ETFs importantes para identificación de regímenes
            additional_tickers = ['^VIX', 'TLT', 'IEF', 'HYG', 'LQD']  # VIX, bonos largos, bonos intermedios, HY, IG
            
            # Incluir índice de mercado y tickers adicionales
            tickers = self.sp500_stocks + [self.market_index] + additional_tickers
            
            print(f"Descargando datos para {len(tickers)} activos desde {extended_start} hasta {self.end_date}...")
            
            # Descargar datos de precio y volumen
            self.data = yf.download(
                tickers, 
                start=extended_start, 
                end=self.end_date,
                group_by='ticker',
                progress=False
            )
            
            # Separar datos del mercado
            self.market_data = pd.DataFrame({
                'close': self.data[self.market_index]['Close'],
                'high': self.data[self.market_index]['High'],
                'low': self.data[self.market_index]['Low'],
                'volume': self.data[self.market_index]['Volume']
            })
            
            # Calcular retornos diarios del mercado
            self.market_data['returns'] = self.market_data['close'].pct_change()
            
            # MEJORA 5: Guardar datos del VIX para identificación de regímenes
            if '^VIX' in self.data.columns.levels[0]:
                self.vix_data = pd.DataFrame({
                    'close': self.data['^VIX']['Close'],
                    'high': self.data['^VIX']['High'],
                    'low': self.data['^VIX']['Low']
                })
            
            # Limpiar datos de precio y volumen para acciones individuales
            stocks_data = {}
            
            for ticker in tqdm(self.sp500_stocks, desc="Procesando activos"):
                try:
                    if ticker in self.data.columns.levels[0]:
                        stock_data = pd.DataFrame({
                            'close': self.data[ticker]['Close'],
                            'open': self.data[ticker]['Open'],
                            'high': self.data[ticker]['High'],
                            'low': self.data[ticker]['Low'],
                            'volume': self.data[ticker]['Volume'],
                        })
                        
                        # Calcular retornos diarios
                        stock_data['returns'] = stock_data['close'].pct_change()
                        
                        # Solo mantener acciones con suficientes datos
                        if stock_data['close'].dropna().shape[0] > 252:  # Al menos un año de datos
                            stocks_data[ticker] = stock_data
                except Exception as e:
                    logging.error(f"Error procesando {ticker}: {str(e)}")
            
            # Guardar datos procesados
            self.stock_data = stocks_data
            
            print(f"Datos descargados y procesados para {len(stocks_data)} activos válidos.")
                
        except Exception as e:
            logging.error(f"Error descargando datos: {str(e)}")
            raise
    
    def calculate_market_regime_features(self) -> pd.DataFrame:
        """
        Calcula características mejoradas para la identificación de regímenes de mercado.
        
        Returns:
            DataFrame con características para identificación de regímenes
        """
        market = self.market_data.copy()
        
        # MEJORA 6: Características más relevantes para identificación de regímenes
        features = pd.DataFrame(index=market.index)
        
        # 1. Tendencia de precios (corto, medio y largo plazo)
        market['sma20'] = market['close'].rolling(20).mean()
        market['sma50'] = market['close'].rolling(50).mean()
        market['sma200'] = market['close'].rolling(200).mean()
        
        features['trend_st'] = market['close'] / market['sma20'] - 1  # Tendencia corto plazo
        features['trend_mt'] = market['close'] / market['sma50'] - 1  # Tendencia medio plazo
        features['trend_lt'] = market['close'] / market['sma200'] - 1  # Tendencia largo plazo
        
        # 2. Volatilidad en diferentes períodos
        features['volatility_st'] = market['returns'].rolling(21).std() * np.sqrt(252)  # Volatilidad 1 mes
        features['volatility_mt'] = market['returns'].rolling(63).std() * np.sqrt(252)  # Volatilidad 3 meses
        
        # 3. Usar VIX si está disponible
        if self.vix_data is not None:
            vix = self.vix_data.copy()
            vix_index = vix.index.intersection(features.index)
            features.loc[vix_index, 'vix'] = vix.loc[vix_index, 'close']
            features['vix_ma20'] = features['vix'].rolling(20).mean()
            features['vix_ratio'] = features['vix'] / features['vix_ma20']
        
        # 4. Momentum de mercado (rendimiento de diferentes períodos)
        features['momentum_1m'] = market['close'].pct_change(21)  # 1 mes
        features['momentum_3m'] = market['close'].pct_change(63)  # 3 meses
        features['momentum_6m'] = market['close'].pct_change(126)  # 6 meses
        
        # 5. Amplitud de mercado (usamos volatilidad como proxy)
        features['market_breadth'] = features['volatility_mt'] / features['volatility_st']
        
        # Imputar valores faltantes con la media
        features = features.fillna(features.mean())
        
        return features
    
    def identify_market_regimes(self, training_data: pd.DataFrame) -> np.ndarray:
        """
        Identifica regímenes de mercado utilizando GMM con selección de características.
        
        Args:
            training_data: DataFrame con características para entrenamiento
        
        Returns:
            Array con etiquetas de régimen
        """
        # MEJORA 7: Selección de variables más importantes para evitar ruido
        # Usamos un conjunto más pequeño de variables para obtener regímenes más estables
        key_features = ['volatility_st', 'trend_mt', 'momentum_3m']
        
        if 'vix' in training_data.columns:
            key_features.append('vix_ratio')
            
        # Extraer características clave
        regime_data = training_data[key_features].copy()
        
        # Normalizar características
        scaler = StandardScaler()
        normalized_data = scaler.fit_transform(regime_data)
        
        # MEJORA 8: Inicialización más robusta para GMM
        # Ajustar GMM con múltiples inicializaciones
        gmm = GaussianMixture(
            n_components=self.n_regimes,
            covariance_type='full',
            random_state=self.random_state,
            n_init=20,  # Aumentado para mayor estabilidad
            max_iter=200
        )
        
        gmm.fit(normalized_data)
        labels = gmm.predict(normalized_data)
        
        # MEJORA 9: Ordenar regímenes por volatilidad para mejor interpretación
        # Calcular volatilidad media en cada régimen
        regime_volatility = {}
        for i in range(self.n_regimes):
            mask = (labels == i)
            if mask.sum() > 0:
                regime_volatility[i] = training_data.loc[mask, 'volatility_st'].mean()
        
        # Ordenar regímenes por volatilidad (0: baja, 1: media, 2: alta)
        sorted_regimes = sorted(regime_volatility.items(), key=lambda x: x[1])
        regime_map = {old: new for new, (old, _) in enumerate(sorted_regimes)}
        
        # Reasignar etiquetas
        new_labels = np.array([regime_map[label] for label in labels])
        
        return new_labels
    
    def identify_regimes(self):
        """
        Identifica regímenes de mercado para todo el período.
        Actualiza el modelo en intervalos regulares para evitar look-ahead bias.
        """
        print("Identificando regímenes de mercado...")
        
        # Calcular características para todo el período
        all_features = self.calculate_market_regime_features()
        
        # Obtener fechas para actualizaciones del modelo
        if self.regime_update_frequency == 'M':
            update_dates = pd.date_range(start=self.start_date, end=self.end_date, freq='M')
        else:
            update_dates = pd.date_range(start=self.start_date, end=self.end_date, freq=self.regime_update_frequency)
        
        # Añadir fecha inicial si no está incluida
        if update_dates[0] > pd.Timestamp(self.start_date):
            update_dates = pd.DatetimeIndex([pd.Timestamp(self.start_date)]).append(update_dates)
        
        # DataFrame para almacenar regímenes
        self.regimes = pd.DataFrame(index=all_features.index, columns=['regime'])
        
        # Para cada fecha de actualización, entrenar modelo con datos disponibles hasta ese momento
        for i in range(len(update_dates) - 1):
            current_date = update_dates[i]
            next_date = update_dates[i + 1]
            
            # Obtener datos de entrenamiento (solo datos hasta current_date)
            lookback_start = current_date - pd.Timedelta(days=int(365.25 * self.lookback_years))
            train_features = all_features.loc[
                (all_features.index >= lookback_start) & 
                (all_features.index <= current_date)
            ]
            
            if train_features.shape[0] > 30:  # Asegurar suficientes datos
                # Identificar regímenes
                labels = self.identify_market_regimes(train_features)
                
                # Aplicar el modelo a datos entre current_date y next_date
                predict_features = all_features.loc[
                    (all_features.index > current_date) & 
                    (all_features.index <= next_date)
                ]
                
                if predict_features.shape[0] > 0:
                    # MEJORA 10: Selección de características consistente
                    key_features = ['volatility_st', 'trend_mt', 'momentum_3m']
                    if 'vix' in train_features.columns:
                        key_features.append('vix_ratio')
                    
                    # Normalizar con el mismo scaler
                    scaler = StandardScaler()
                    train_key_features = train_features[key_features]
                    scaler.fit(train_key_features)
                    predict_key_features = predict_features[key_features]
                    predict_normalized = scaler.transform(predict_key_features)
                    
                    # Ajustar GMM nuevamente con datos de entrenamiento
                    gmm = GaussianMixture(
                        n_components=self.n_regimes,
                        covariance_type='full',
                        random_state=self.random_state,
                        n_init=20,
                        max_iter=200
                    )
                    gmm.fit(scaler.transform(train_key_features))
                    
                    # Predecir regímenes
                    predict_labels = gmm.predict(predict_normalized)
                    
                    # MEJORA 11: Aplicar la misma reordenación por volatilidad
                    regime_volatility = {}
                    for j in range(self.n_regimes):
                        mask = (labels == j)
                        if mask.sum() > 0:
                            regime_volatility[j] = train_features.loc[mask, 'volatility_st'].mean()
                    
                    sorted_regimes = sorted(regime_volatility.items(), key=lambda x: x[1])
                    regime_map = {old: new for new, (old, _) in enumerate(sorted_regimes)}
                    
                    # Guardar predicciones con etiquetas mapeadas
                    predict_dates = predict_features.index
                    for j, date in enumerate(predict_dates):
                        old_label = predict_labels[j]
                        if old_label in regime_map:
                            self.regimes.loc[date, 'regime'] = regime_map[old_label]
                        else:
                            self.regimes.loc[date, 'regime'] = old_label
        
        # Llenar valores faltantes forward fill
        self.regimes = self.regimes.fillna(method='ffill')
        
        # Si hay valores faltantes al inicio, usar backfill
        self.regimes = self.regimes.fillna(method='bfill')
        
        # Convertir a enteros
        self.regimes['regime'] = self.regimes['regime'].astype(int)
        
        # Guardar regímenes identificados
        self.regimes.to_csv('./artifacts/results/data/market_regimes.csv')
        
        # Visualizar regímenes
        self.plot_market_regimes()
    
    def plot_market_regimes(self):
        """
        Visualiza los regímenes de mercado identificados junto con el precio del índice.
        """
        if self.regimes is None:
            print("No hay regímenes identificados para visualizar.")
            return
        
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
        
        # Gráfico de precios
        ax1.plot(self.market_data.loc[self.regimes.index, 'close'], 'k-', label='Índice de Mercado')
        ax1.set_ylabel('Precio')
        ax1.set_title('Regímenes de Mercado Identificados')
        ax1.legend(loc='upper left')
        
        # MEJORA 12: Añadir gráfico de VIX si disponible
        if self.vix_data is not None:
            common_dates = self.vix_data.index.intersection(self.regimes.index)
            ax2.plot(common_dates, self.vix_data.loc[common_dates, 'close'], 'r-', label='VIX')
            ax2.set_ylabel('VIX')
            ax2.legend(loc='upper left')
        else:
            # Si no hay VIX, mostrar volatilidad
            vol_data = self.market_data['returns'].rolling(21).std() * np.sqrt(252)
            ax2.plot(vol_data.loc[self.regimes.index], 'r-', label='Volatilidad (21d)')
            ax2.set_ylabel('Volatilidad Anualizada')
            ax2.legend(loc='upper left')
        
        # Gráfico de regímenes
        regime_colors = ['green', 'orange', 'red']  # Bajo, medio, alto riesgo
        
        for regime in range(self.n_regimes):
            regime_periods = self.regimes[self.regimes['regime'] == regime].index
            
            if len(regime_periods) > 0:
                # Agrupar períodos consecutivos
                groups = []
                current_group = [regime_periods[0]]
                
                for i in range(1, len(regime_periods)):
                    if (regime_periods[i] - regime_periods[i-1]).days <= 2:  # Considerar días consecutivos
                        current_group.append(regime_periods[i])
                    else:
                        groups.append(current_group)
                        current_group = [regime_periods[i]]
                
                groups.append(current_group)
                
                # Dibujar rectángulos para cada grupo
                for group in groups:
                    if len(group) > 1:
                        start_date = group[0]
                        end_date = group[-1]
                        ax3.axvspan(start_date, end_date, alpha=0.3, color=regime_colors[regime])
        
        # Etiquetar regímenes
        labels = ['Baja Volatilidad/Alcista', 'Transición/Neutral', 'Alta Volatilidad/Bajista']
        handles = [plt.Rectangle((0,0),1,1, color=regime_colors[i], alpha=0.3) for i in range(self.n_regimes)]
        ax3.legend(handles, labels, loc='upper left')
        
        ax3.plot(self.regimes.index, self.regimes['regime'], 'k-', alpha=0.7)
        ax3.set_ylabel('Régimen')
        ax3.set_xlabel('Fecha')
        
        # Ajustar formato de fechas
        fig.autofmt_xdate()
        
        plt.tight_layout()
        plt.savefig('./artifacts/results/figures/market_regimes.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    def calculate_factors(self, date: pd.Timestamp) -> Dict[str, pd.Series]:
        """
        Calcula todos los factores para una fecha específica.
        
        Args:
            date: Fecha para la que se calculan los factores
        
        Returns:
            Diccionario con Series de factores calculados
        """
        # MEJORA 13: Ventana de datos variable según factor
        # Usar más datos históricos para factores de value y quality
        lookback_long = date - pd.Timedelta(days=365*2)  # 2 años para factores de más largo plazo
        lookback_medium = date - pd.Timedelta(days=365)  # 1 año para factores generales
        lookback_short = date - pd.Timedelta(days=90)    # 3 meses para momentum corto plazo y reversal
        
        # Filtrar acciones con datos completos
        valid_stocks = []
        for ticker, data in self.stock_data.items():
            slice_data = data.loc[lookback_medium:date]
            if slice_data.shape[0] > 200:  # Al menos 200 días de datos
                valid_stocks.append(ticker)
        
        factors = {}
        
        # 1. VALOR
        # Para factores fundamentales, en una implementación real usaríamos datos trimestrales
        # Aquí usamos proxies basados en precio y volumen
        
        # MEJORA 14: Mejorar proxy de valor usando datos de precio históricos
        pb_proxy = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_long:date]
            if data.shape[0] > 0:
                # Usar ratio precio actual / precio máximo histórico como proxy de valor
                current_price = data['close'].iloc[-1]
                historical_high = data['close'].max()
                if historical_high > 0:
                    pb_proxy[ticker] = current_price / historical_high
        factors['PB_proxy'] = pd.Series(pb_proxy)
        
        # MEJORA 15: Proxy de valor basado en volumen-precio
        eveb_proxy = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_long:date]
            if data.shape[0] > 0:
                # Precio / (volumen * volatilidad) como proxy de valor
                avg_volume = data['volume'].mean()
                volatility = data['returns'].std() * np.sqrt(252)
                current_price = data['close'].iloc[-1]
                if avg_volume > 0 and volatility > 0:
                    eveb_proxy[ticker] = current_price / (avg_volume * volatility)
        factors['EVEB_proxy'] = pd.Series(eveb_proxy)
        
        # 2. MOMENTUM
        
        # MEJORA 16: Múltiples períodos de momentum con ponderación
        momentum_periods = [
            (21, 0.2),    # 1 mes (20%)
            (63, 0.3),    # 3 meses (30%)
            (126, 0.3),   # 6 meses (30%)
            (252, 0.2)    # 12 meses (20%)
        ]
        
        weighted_momentum = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_long:date]
            if data.shape[0] > 252:  # Al menos 1 año de datos
                momentum_sum = 0.0
                weight_sum = 0.0
                
                for days, weight in momentum_periods:
                    if data.shape[0] >= days:
                        # Precio actual vs hace "days" días
                        ret = data['close'].iloc[-1] / data['close'].iloc[-days] - 1
                        momentum_sum += ret * weight
                        weight_sum += weight
                
                if weight_sum > 0:
                    weighted_momentum[ticker] = momentum_sum / weight_sum
                    
        factors['Momentum_weighted'] = pd.Series(weighted_momentum)
        
        # MEJORA 17: Momentum ajustado por volatilidad (Sharpe de momentum)
        momentum_sharpe = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_medium:date]
            if data.shape[0] > 125:  # Al menos 6 meses de datos
                # Rendimiento de 6 meses
                ret_6m = data['close'].iloc[-1] / data['close'].iloc[-min(125, data.shape[0])] - 1
                
                # Volatilidad de 6 meses
                vol_6m = data['returns'].tail(min(125, data.shape[0])).std() * np.sqrt(252)
                
                if vol_6m > 0:
                    momentum_sharpe[ticker] = ret_6m / vol_6m
                    
        factors['Momentum_sharpe'] = pd.Series(momentum_sharpe)
        
        # 3. CALIDAD
        
        # MEJORA 18: Mejor proxy para estabilidad de ganancias
        price_stability = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_long:date]
            if data.shape[0] > 252:
                # Calcular downside deviation (solo rendimientos negativos)
                neg_returns = data['returns'][data['returns'] < 0]
                if len(neg_returns) > 0:
                    downside_vol = neg_returns.std() * np.sqrt(252)
                    if downside_vol > 0:
                        price_stability[ticker] = 1 / downside_vol
                        
        factors['Downside_stability'] = pd.Series(price_stability)
        
        # MEJORA 19: Proxy de calidad basado en consistencia de rendimientos
        consistency = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_long:date]
            if data.shape[0] > 252:
                # Calcular % de días con rendimiento positivo
                positive_days = (data['returns'] > 0).sum() / len(data['returns'])
                consistency[ticker] = positive_days
                
        factors['Return_consistency'] = pd.Series(consistency)
        
        # 4. VOLATILIDAD
        
        # MEJORA 20: Múltiples medidas de volatilidad y riesgo
        
        # Volatilidad realizada (más reciente tiene mayor peso)
        vol_weights = [0.6, 0.3, 0.1]  # 60% 1 mes, 30% 3 meses, 10% 6 meses
        periods = [21, 63, 126]
        
        weighted_vol = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_medium:date]
            if data.shape[0] > max(periods):
                vol_sum = 0.0
                weight_sum = 0.0
                
                for days, weight in zip(periods, vol_weights):
                    period_vol = data['returns'].tail(days).std() * np.sqrt(252)
                    vol_sum += period_vol * weight
                    weight_sum += weight
                
                if weight_sum > 0:
                    weighted_vol[ticker] = vol_sum / weight_sum
                    
        factors['Volatility_weighted'] = pd.Series(weighted_vol)
        
        # Beta mejorado (más estable)
        beta_improved = {}
        for ticker in valid_stocks:
            stock_data = self.stock_data[ticker].loc[lookback_medium:date]
            market_slice = self.market_data.loc[lookback_medium:date]
            
            # Alinear datos
            common_dates = stock_data.index.intersection(market_slice.index)
            if len(common_dates) > 120:  # Al menos 6 meses de datos
                stock_returns = stock_data.loc[common_dates, 'returns']
                market_returns = market_slice.loc[common_dates, 'returns']
                
                # Remover NaN
                valid_mask = ~(np.isnan(stock_returns) | np.isnan(market_returns))
                stock_returns = stock_returns[valid_mask]
                market_returns = market_returns[valid_mask]
                
                if len(stock_returns) > 120:
                    # MEJORA 21: Beta con ponderación exponencial (mayor peso a datos recientes)
                    # Crear pesos exponenciales (más reciente = más peso)
                    n = len(stock_returns)
                    weights_recent = np.exp(np.linspace(0, 1, n)) - 1  # Pesos exponenciales
                    weights_recent = weights_recent / weights_recent.sum()  # Normalizar
                    
                    # Calcular beta ponderado
                    # CORRECCIÓN: Usar implementación correcta de covarianza y varianza ponderadas
                    # Calcular media ponderada
                    weighted_mean_market = np.sum(market_returns * weights_recent)
                    weighted_mean_stock = np.sum(stock_returns * weights_recent)
                    
                    # Calcular covarianza ponderada
                    cov = np.sum(weights_recent * (stock_returns - weighted_mean_stock) * (market_returns - weighted_mean_market))
                    
                    # Calcular varianza ponderada
                    var = np.sum(weights_recent * (market_returns - weighted_mean_market)**2)
                    
                    if var > 0:
                        beta = cov / var
                        beta_improved[ticker] = beta
                        
        factors['Beta_improved'] = pd.Series(beta_improved)
        
        # 5. LIQUIDEZ
        
        # MEJORA 22: Tendencia de volumen
        vol_trend = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_medium:date]
            if data.shape[0] > 60:
                # Calcular tendencia de volumen: volumen reciente vs histórico
                recent_vol = data['volume'].tail(20).mean()
                historical_vol = data['volume'].mean()
                
                if historical_vol > 0:
                    vol_trend[ticker] = recent_vol / historical_vol
                    
        factors['Volume_trend'] = pd.Series(vol_trend)
        
        # MEJORA 23: Mejor ratio de iliquidez
        amihud_improved = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_medium:date]
            if data.shape[0] > 60:
                # Solo considerar días con volumen significativo
                valid_days = data[data['volume'] > 0]
                if len(valid_days) > 30:
                    # MEJORA: Ratio de Amihud mejorado (|return| / (price * volume))
                    price_vol = valid_days['close'] * valid_days['volume']
                    illiq = (valid_days['returns'].abs() / price_vol).mean() * 1e9  # Escalar
                    if illiq > 0:
                        amihud_improved[ticker] = illiq
                        
        factors['Illiquidity_improved'] = pd.Series(amihud_improved)
        
        # 6. REVERSAL (Nuevo factor)
        
        # MEJORA 24: Añadir factor de reversión a corto plazo
        reversal_st = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_short:date]
            if data.shape[0] > 20:
                # Retorno 1 semana (invertido para capturar reversión)
                ret_1w = data['close'].iloc[-1] / data['close'].iloc[-min(5, data.shape[0])] - 1
                reversal_st[ticker] = -ret_1w  # Invertir para que valores positivos indiquen potencial de reversión
                
        factors['Reversal_1w'] = pd.Series(reversal_st)
        
        # MEJORA 25: Reversión respecto a media móvil
        reversal_ma = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_short:date]
            if data.shape[0] > 20:
                # Calcular desviación de precio respecto a media móvil de 20 días
                if not data['close'].empty:
                    ma20 = data['close'].rolling(20).mean().iloc[-1]
                    current = data['close'].iloc[-1]
                    if ma20 > 0:
                        # Negativo si precio > MA (potencial reversión a la baja)
                        # Positivo si precio < MA (potencial reversión al alza)
                        reversal_ma[ticker] = (ma20 / current) - 1
                        
        factors['Reversal_ma20'] = pd.Series(reversal_ma)
        
        # Normalizar todos los factores (z-score) y controlar outliers
        normalized_factors = {}
        for factor_name, factor_values in factors.items():
            if len(factor_values) > 0:
                # Winsorizar para controlar outliers (recortar los extremos al 1% y 99%)
                lower_bound = factor_values.quantile(0.01)
                upper_bound = factor_values.quantile(0.99)
                winsorized = factor_values.clip(lower=lower_bound, upper=upper_bound)
                
                # Z-score normalization
                mean = winsorized.mean()
                std = winsorized.std()
                if std > 0:
                    normalized = (winsorized - mean) / std
                    normalized_factors[factor_name] = normalized
        
        # Invertir factores donde valores menores son mejores
        for factor_name in ['EVEB_proxy', 'Volatility_weighted', 'Illiquidity_improved']:
            if factor_name in normalized_factors:
                normalized_factors[factor_name] = -normalized_factors[factor_name]
        
        # MEJORA 26: Organizar factores por categoría con pesos internos mejorados
        factor_by_category = {
            'Value': [('PB_proxy', 0.5), ('EVEB_proxy', 0.5)],
            'Momentum': [('Momentum_weighted', 0.6), ('Momentum_sharpe', 0.4)],
            'Quality': [('Downside_stability', 0.6), ('Return_consistency', 0.4)],
            'Volatility': [('Volatility_weighted', 0.4), ('Beta_improved', 0.6)],
            'Liquidity': [('Volume_trend', 0.5), ('Illiquidity_improved', 0.5)],
            'Reversal': [('Reversal_1w', 0.5), ('Reversal_ma20', 0.5)]
        }
        
        # Asegurar que todas las acciones tengan datos en todas las categorías
        common_stocks = set()
        for factor in normalized_factors.values():
            if common_stocks:
                common_stocks = common_stocks.intersection(set(factor.index))
            else:
                common_stocks = set(factor.index)
        
        # Filtrar a acciones comunes
        filtered_factors = {}
        for factor_name, factor_values in normalized_factors.items():
            filtered_factors[factor_name] = factor_values.loc[list(common_stocks)]
        
        return filtered_factors, factor_by_category
    
    def thompson_sampling(self, regime: int) -> Dict[str, float]:
        """
        Implementa Thompson Sampling para seleccionar categorías de factores
        basado en sus distribuciones de creencia.
        
        Args:
            regime: Régimen de mercado actual
        
        Returns:
            Diccionario con pesos para cada categoría
        """
        samples = {}
        
        # MEJORA 27: Ajuste de pesos según régimen de mercado
        # Aplicar factores de ajuste para reforzar ciertas categorías en ciertos regímenes
        regime_adjustments = {
            0: {  # Régimen 0: Baja volatilidad/alcista
                'Value': 1.0,
                'Momentum': 1.2,  # Potenciar momentum en mercados alcistas
                'Quality': 0.8,
                'Volatility': 0.8,
                'Liquidity': 0.8,
                'Reversal': 0.8
            },
            1: {  # Régimen 1: Transición/neutral
                'Value': 1.0,
                'Momentum': 1.0,
                'Quality': 1.0,
                'Volatility': 1.0,
                'Liquidity': 1.0,
                'Reversal': 1.0
            },
            2: {  # Régimen 2: Alta volatilidad/bajista
                'Value': 0.8,
                'Momentum': 0.8,
                'Quality': 1.2,  # Potenciar calidad en mercados bajistas
                'Volatility': 1.0,
                'Liquidity': 1.1,
                'Reversal': 1.2  # Potenciar reversión en mercados volátiles
            }
        }
        
        # Para cada categoría, muestrear de su distribución Beta con ajuste
        for category in self.factor_categories:
            alpha = self.belief_distributions[category][regime]['alpha']
            beta = self.belief_distributions[category][regime]['beta']
            
            # MEJORA 28: Tomar múltiples muestras y promediar para más estabilidad
            n_samples = 5
            category_samples = []
            
            for _ in range(n_samples):
                sample = np.random.beta(alpha, beta)
                category_samples.append(sample)
            
            # Usar promedio de muestras
            avg_sample = np.mean(category_samples)
            
            # Aplicar ajuste de régimen
            adjustment = regime_adjustments.get(regime, {}).get(category, 1.0)
            samples[category] = avg_sample * adjustment
        
        # Normalizar para obtener pesos
        total = sum(samples.values())
        weights = {category: sample / total for category, sample in samples.items()}
        
        return weights
    
    def update_beliefs(self, date: pd.Timestamp, lookback_days: int = 30):
        """
        Actualiza las distribuciones de creencia basado en rendimiento reciente
        de cada categoría en cada régimen.
        
        Args:
            date: Fecha actual
            lookback_days: Días hacia atrás para evaluar rendimiento
        """
        # Calcular ventana de tiempo
        start_date = date - pd.Timedelta(days=lookback_days)
        
        # Obtener regímenes en el período
        if self.regimes is None or start_date not in self.regimes.index:
            return
            
        regime_period = self.regimes.loc[start_date:date]
        
        # Para cada régimen presente en el período
        for regime in regime_period['regime'].unique():
            regime_dates = regime_period[regime_period['regime'] == regime].index
            if len(regime_dates) < 5:  # Necesitamos suficientes días en el régimen
                continue
                
            # Para cada categoría, evaluar rendimiento
            for category in self.factor_categories:
                # Suponiendo que tenemos un método para evaluar rendimiento de categoría
                performance = self._evaluate_category_performance(category, regime_dates)
                
                # Actualizar distribuciones de creencia
                if performance is not None:
                    # MEJORA 29: Mejorar actualización Bayesiana con escala de rendimiento
                    # En lugar de éxito/fracaso binario, usar una escala continua
                    performance_score = (performance + 1) / 2  # Convertir a [0, 1]
                    
                    # Limitar valores extremos
                    performance_score = max(0.1, min(0.9, performance_score))
                    
                    # Factores de actualización proporcionales al rendimiento
                    success_update = performance_score * 2  # Máx 1.8
                    failure_update = (1 - performance_score) * 2  # Máx 1.8
                    
                    # Actualizar distribución de creencia (prior conjugado Beta)
                    self.belief_distributions[category][regime]['alpha'] += success_update
                    self.belief_distributions[category][regime]['beta'] += failure_update
                    
                    # Aplicar decay exponencial para dar más peso a observaciones recientes
                    decay_factor = 0.95
                    alpha = self.belief_distributions[category][regime]['alpha']
                    beta = self.belief_distributions[category][regime]['beta']
                    
                    # Decay
                    self.belief_distributions[category][regime]['alpha'] = 1 + decay_factor * (alpha - 1)
                    self.belief_distributions[category][regime]['beta'] = 1 + decay_factor * (beta - 1)
    
    def _evaluate_category_performance(self, category: str, dates: pd.DatetimeIndex) -> Optional[float]:
        """
        Evalúa el rendimiento de una categoría de factores en un período específico.
        
        Args:
            category: Categoría de factores
            dates: Fechas para evaluar rendimiento
        
        Returns:
            Score de rendimiento (escala -1 a 1)
        """
        # MEJORA 30: Evaluación de rendimiento más realista
        
        # Si no tenemos suficientes fechas, retornar None
        if len(dates) < 5:
            return None
            
        # Verificar si tenemos datos de rendimiento previos
        if category in self.factor_performance and len(self.factor_performance[category]) > 0:
            # Filtrar a fechas relevantes
            perf = [p for d, p in self.factor_performance[category] if d in dates]
            if len(perf) > 0:
                return np.mean(perf)
        
        # Si no hay datos previos, usar simulación más realista basada en regímenes conocidos
        regime = self.regimes.loc[dates[0], 'regime']
        
        # MEJORA 31: Simulación de rendimiento basada en literatura académica
        # Valores basados en estudios sobre factores en diferentes regímenes
        expected_performances = {
            0: {  # Régimen 0: Baja volatilidad/alcista
                'Value': 0.1,       # Value tiende a funcionar mal en mercados alcistas
                'Momentum': 0.5,    # Momentum funciona bien en mercados alcistas
                'Quality': 0.3,     # Quality es neutral-positivo
                'Volatility': 0.2,  # Volatilidad baja es positiva pero no extraordinaria
                'Liquidity': 0.1,   # Liquidez menos importante en mercados tranquilos
                'Reversal': -0.1    # Reversión funciona mal en tendencias fuertes
            },
            1: {  # Régimen 1: Transición/neutral
                'Value': 0.3,       # Value funciona mejor en transiciones
                'Momentum': 0.2,    # Momentum es menos efectivo en transiciones
                'Quality': 0.4,     # Quality es consistente
                'Volatility': 0.3,  # Volatilidad controlada es positiva
                'Liquidity': 0.2,   # Liquidez más importante en transiciones
                'Reversal': 0.3     # Reversión funciona en mercados laterales
            },
            2: {  # Régimen 2: Alta volatilidad/bajista
                'Value': 0.4,       # Value puede funcionar en crisis (pero con lag)
                'Momentum': -0.2,   # Momentum sufre en mercados volátiles
                'Quality': 0.5,     # Quality es defensivo
                'Volatility': 0.4,  # Baja volatilidad es muy valiosa
                'Liquidity': 0.4,   # Liquidez es crítica
                'Reversal': 0.4     # Reversión funciona muy bien en volatilidad
            }
        }
        
        # Obtener rendimiento esperado para esta categoría en este régimen
        base_performance = expected_performances.get(regime, {}).get(category, 0)
        
        # Añadir ruido aleatorio más acotado
        noise = np.random.normal(0, 0.1)  # Desviación estándar reducida
        
        return max(-1.0, min(1.0, base_performance + noise))  # Limitar a [-1, 1]
    
    def calculate_factor_scores(self, factors: Dict[str, pd.Series], factor_by_category: Dict[str, List[Tuple[str, float]]]) -> Dict[str, pd.Series]:
        """
        Calcula scores compuestos para cada categoría de factores.
        
        Args:
            factors: Diccionario de factores individuales
            factor_by_category: Mapping de categorías a factores con pesos
            
        Returns:
            Diccionario con scores para cada categoría
        """
        category_scores = {}
        
        # MEJORA 32: Usar pesos internos para cada factor en su categoría
        for category, factor_items in factor_by_category.items():
            available_items = [(f, w) for f, w in factor_items if f in factors]
            
            if available_items:
                # Calcular suma ponderada de factores
                scores = None
                total_weight = 0
                
                for factor_name, weight in available_items:
                    if scores is None:
                        scores = factors[factor_name] * weight
                    else:
                        scores += factors[factor_name] * weight
                    total_weight += weight
                
                if total_weight > 0 and scores is not None:
                    category_scores[category] = scores / total_weight
        
        return category_scores
    
    def construct_portfolio(self, date: pd.Timestamp) -> pd.Series:
        """
        Construye el portafolio para una fecha específica utilizando
        el enfoque de aprendizaje por refuerzo.
        
        Args:
            date: Fecha para la que se construye el portafolio
            
        Returns:
            Series con pesos del portafolio
        """
        # Verificar si tenemos identificación de régimen para esta fecha
        if self.regimes is None or date not in self.regimes.index:
            print(f"No hay datos de régimen para la fecha {date}. Usando equiponderación.")
            # Usar equiponderación entre categorías si no hay datos de régimen
            category_weights = {cat: 1.0/len(self.factor_categories) for cat in self.factor_categories}
        else:
            # Obtener régimen actual
            current_regime = self.regimes.loc[date, 'regime']
            self.current_regime = current_regime
            
            # Usar Thompson Sampling para seleccionar pesos de categorías
            category_weights = self.thompson_sampling(current_regime)
        
        # Calcular factores
        factors, factor_by_category = self.calculate_factors(date)
        
        # Si no hay suficientes factores, retornar portafolio vacío
        if not factors or len(factors) == 0:
            return pd.Series(dtype='float64')
        
        # Calcular scores por categoría
        category_scores = self.calculate_factor_scores(factors, factor_by_category)
        
        # MEJORA 33: Combinar scores con pesos adaptativos
        # Ajustar ligeramente pesos según historial reciente
        if hasattr(self, 'previous_weights') and self.previous_weights is not None:
            # Obtener rendimientos recientes
            lookback_start = date - pd.Timedelta(days=30)
            recent_returns = {}
            
            for ticker in self.previous_weights.index:
                if ticker in self.stock_data and lookback_start in self.stock_data[ticker].index:
                    recent_returns[ticker] = self.stock_data[ticker].loc[lookback_start:date, 'returns'].mean() * 21  # Mensualizado
            
            if recent_returns:
                # Calcular rendimiento reciente ponderado del portafolio anterior
                recent_portfolio_return = sum(self.previous_weights.get(ticker, 0) * recent_returns.get(ticker, 0) 
                                             for ticker in self.previous_weights.index)
                
                # Ajustar ligeramente hacia factores que funcionaron bien
                if recent_portfolio_return > 0.01:  # Si rendimiento mensualizado > 1%
                    # Dar más peso a factores que produjeron mejores resultados
                    for category in self.factor_categories:
                        if category in category_weights:
                            category_weights[category] *= (1 + min(0.2, recent_portfolio_return * 2))  # Límite de 20% aumento
                    
                    # Renormalizar
                    total = sum(category_weights.values())
                    category_weights = {k: v/total for k, v in category_weights.items()}
        
        # Combinar scores ponderados por los pesos de categorías
        combined_score = pd.Series(0, index=next(iter(factors.values())).index)
        
        for category, weight in category_weights.items():
            if category in category_scores:
                combined_score += category_scores[category] * weight
        
        # Normalizar scores
        combined_score = (combined_score - combined_score.mean()) / combined_score.std()
        
        # Rankear acciones
        ranked_stocks = combined_score.sort_values(ascending=False)
        
        # MEJORA 34: Selección de stocks más óptima (cuantiles en lugar de umbral fijo)
        # Usar selección dinámica basada en distribución de scores
        upper_quantile = ranked_stocks.quantile(0.7)  # Top 30%
        selected_stocks = ranked_stocks[ranked_stocks >= upper_quantile].index
        
        # Asignar pesos usando función sigmoide para suavizar transición 
        weights = pd.Series(0, index=ranked_stocks.index)
        for stock in selected_stocks:
            # Convertir score a peso usando función sigmoide
            score = ranked_stocks[stock]
            weight = 1 / (1 + np.exp(-2 * (score - upper_quantile)))  # Sigmoide centrada en upper_quantile
            weights[stock] = weight
        
        # Normalizar pesos
        if weights.sum() > 0:
            weights = weights / weights.sum()
        
        # Ajustar pesos para cumplir restricciones
        weights = self._apply_constraints(weights, date)
        
        return weights
    
    def _apply_constraints(self, weights: pd.Series, date: pd.Timestamp) -> pd.Series:
        """
        Aplica restricciones de portafolio: beta neutral, límites de concentración, etc.
        
        Args:
            weights: Pesos iniciales del portafolio
            date: Fecha actual
            
        Returns:
            Pesos ajustados
        """
        # Si no hay pesos positivos, retornar serie vacía
        if weights.sum() == 0:
            return weights
            
        # Recortar a máximo peso por acción
        weights = weights.clip(upper=self.max_stock_weight)
        
        # Renormalizar
        if weights.sum() > 0:
            weights = weights / weights.sum()
        
        # MEJORA 35: Ajuste de beta más robusto
        try:
            # Calcular betas
            betas = {}
            lookback_start = date - pd.Timedelta(days=252)
            
            for ticker in weights.index[weights > 0]:
                if ticker in self.stock_data:
                    stock_data = self.stock_data[ticker].loc[lookback_start:date]
                    market_slice = self.market_data.loc[lookback_start:date]
                    
                    common_dates = stock_data.index.intersection(market_slice.index)
                    if len(common_dates) > 60:
                        stock_returns = stock_data.loc[common_dates, 'returns']
                        market_returns = market_slice.loc[common_dates, 'returns']
                        
                        # Remover NaN
                        valid_mask = ~(np.isnan(stock_returns) | np.isnan(market_returns))
                        stock_returns = stock_returns[valid_mask]
                        market_returns = market_returns[valid_mask]
                        
                        if len(stock_returns) > 60:
                            # MEJORA: Usar datos recientes con mayor peso
                            n = len(stock_returns)
                            weights_recent = np.exp(np.linspace(0, 1, n)) - 1
                            weights_recent = weights_recent / weights_recent.sum()
                            
                            # CORRECCIÓN: Cálculo de beta ponderado
                            # Calcular medias ponderadas
                            weighted_mean_stock = np.sum(stock_returns * weights_recent)
                            weighted_mean_market = np.sum(market_returns * weights_recent)
                            
                            # Calcular covarianza ponderada
                            cov = np.sum(weights_recent * (stock_returns - weighted_mean_stock) * 
                                        (market_returns - weighted_mean_market))
                            
                            # Calcular varianza ponderada
                            var = np.sum(weights_recent * (market_returns - weighted_mean_market)**2)
                            
                            if var > 0:
                                betas[ticker] = cov / var
            
            # Si tenemos suficientes betas, ajustar portafolio
            if len(betas) > 5:
                beta_series = pd.Series(betas)
                portfolio_beta = (weights.loc[beta_series.index] * beta_series).sum()
                
                # MEJORA 36: Mejor ajuste de beta usando optimización iterativa
                if abs(portfolio_beta - self.target_beta) > self.beta_range:
                    # Ordenar acciones por beta
                    sorted_betas = beta_series.sort_values()
                    
                    iterations = 0
                    max_iterations = 10
                    
                    while abs(portfolio_beta - self.target_beta) > self.beta_range and iterations < max_iterations:
                        if portfolio_beta > self.target_beta + self.beta_range:
                            # Reducir acciones de alto beta, aumentar las de bajo beta
                            # Identificar tickers con betas extremos
                            high_beta_tickers = sorted_betas.tail(int(len(sorted_betas) * 0.3)).index
                            low_beta_tickers = sorted_betas.head(int(len(sorted_betas) * 0.3)).index
                            
                            # Ajustar pesos
                            adjustment = min(0.05, abs(portfolio_beta - self.target_beta) / 5)
                            
                            for ticker in high_beta_tickers:
                                if ticker in weights.index:
                                    weights[ticker] *= (1 - adjustment)
                                    
                            for ticker in low_beta_tickers:
                                if ticker in weights.index:
                                    weights[ticker] *= (1 + adjustment)
                                    
                        elif portfolio_beta < self.target_beta - self.beta_range:
                            # Aumentar acciones de alto beta, reducir las de bajo beta
                            high_beta_tickers = sorted_betas.tail(int(len(sorted_betas) * 0.3)).index
                            low_beta_tickers = sorted_betas.head(int(len(sorted_betas) * 0.3)).index
                            
                            # Ajustar pesos
                            adjustment = min(0.05, abs(portfolio_beta - self.target_beta) / 5)
                            
                            for ticker in high_beta_tickers:
                                if ticker in weights.index:
                                    weights[ticker] *= (1 + adjustment)
                                    
                            for ticker in low_beta_tickers:
                                if ticker in weights.index:
                                    weights[ticker] *= (1 - adjustment)
                        
                        # Renormalizar
                        if weights.sum() > 0:
                            weights = weights / weights.sum()
                        
                        # Recalcular beta de portafolio
                        portfolio_beta = (weights.loc[beta_series.index] * beta_series).sum()
                        iterations += 1
        
        except Exception as e:
            logging.error(f"Error ajustando beta: {str(e)}")
        
        # Renormalizar
        if weights.sum() > 0:
            weights = weights / weights.sum()
        
        # MEJORA 37: Ajuste de volatilidad más preciso
        try:
            # Calcular matriz de varianza-covarianza (simplificada)
            relevant_tickers = [ticker for ticker in weights.index if weights[ticker] > 0]
            
            if len(relevant_tickers) > 5:
                lookback_start = date - pd.Timedelta(days=252)
                returns_data = {}
                
                # Recopilar datos de retornos
                for ticker in relevant_tickers:
                    if ticker in self.stock_data:
                        stock_data = self.stock_data[ticker].loc[lookback_start:date]
                        returns_data[ticker] = stock_data['returns']
                
                # Crear DataFrame de retornos
                returns_df = pd.DataFrame(returns_data)
                returns_df = returns_df.fillna(0)  # Llenar NaNs con ceros
                
                # Calcular volatilidad y correlación
                vol_annual = returns_df.std() * np.sqrt(252)
                corr_matrix = returns_df.corr()
                
                # Calcular volatilidad estimada del portafolio
                portfolio_vol = 0
                for i, ticker_i in enumerate(relevant_tickers):
                    for j, ticker_j in enumerate(relevant_tickers):
                        if ticker_i in vol_annual.index and ticker_j in vol_annual.index:
                            weight_i = weights[ticker_i]
                            weight_j = weights[ticker_j]
                            vol_i = vol_annual[ticker_i]
                            vol_j = vol_annual[ticker_j]
                            corr_ij = corr_matrix.loc[ticker_i, ticker_j]
                            
                            portfolio_vol += weight_i * weight_j * vol_i * vol_j * corr_ij
                
                portfolio_vol = np.sqrt(portfolio_vol)
                
                # Ajustar volatilidad si es necesario
                if portfolio_vol > self.volatility_target * 1.2:  # 20% de margen
                    # Reducir exposición proporcionalmente
                    scale_factor = self.volatility_target / portfolio_vol
                    weights *= scale_factor
                    
                    # El resto en "efectivo" (dejamos los pesos sin sumar 1 para simular efectivo)
                    # En una implementación real, se incluiría posición en activo sin riesgo
                    
        except Exception as e:
            logging.error(f"Error ajustando volatilidad: {str(e)}")
        
        # MEJORA 38: Control de turnover más sofisticado
        if hasattr(self, 'previous_weights') and self.previous_weights is not None:
            # Calcular turnover
            turnover = self._calculate_turnover(self.previous_weights, weights)
            
            # Si el turnover es mayor al máximo, ajustar usando interpolación
            if turnover > self.max_turnover:
                # Calcular ponderación óptima entre portafolio previo y nuevo
                t = self.max_turnover / turnover
                
                # Usar interpolación convexa
                target_weights = t * weights + (1 - t) * self.previous_weights
                
                # Limpiar pesos muy pequeños (< 0.1%)
                target_weights[target_weights < 0.001] = 0
                
                # Renormalizar
                if target_weights.sum() > 0:
                    weights = target_weights / target_weights.sum()
        
        return weights
    
    def _calculate_turnover(self, old_weights: pd.Series, new_weights: pd.Series) -> float:
        """
        Calcula el turnover entre dos conjuntos de pesos.
        
        Args:
            old_weights: Pesos anteriores
            new_weights: Nuevos pesos
            
        Returns:
            Turnover como suma de cambios absolutos / 2
        """
        # Unificar índices
        all_stocks = old_weights.index.union(new_weights.index)
        
        old_unified = pd.Series(0, index=all_stocks)
        new_unified = pd.Series(0, index=all_stocks)
        
        old_unified.loc[old_weights.index] = old_weights
        new_unified.loc[new_weights.index] = new_weights
        
        # Calcular turnover
        turnover = np.sum(np.abs(old_unified - new_unified)) / 2
        
        return turnover
    
    def calculate_portfolio_return(self, weights: pd.Series, date: pd.Timestamp) -> float:
        """
        Calcula el retorno del portafolio para una fecha específica.
        
        Args:
            weights: Pesos del portafolio
            date: Fecha actual
            
        Returns:
            Retorno del portafolio
        """
        next_date = self._get_next_trading_date(date)
        
        if next_date is None:
            return 0.0
        
        portfolio_return = 0.0
        
        for ticker, weight in weights.items():
            if ticker in self.stock_data and next_date in self.stock_data[ticker].index:
                stock_return = self.stock_data[ticker].loc[next_date, 'returns']
                if not np.isnan(stock_return):
                    portfolio_return += weight * stock_return
        
        # Aplicar costo de transacción si tenemos pesos anteriores
        if hasattr(self, 'previous_weights') and self.previous_weights is not None:
            turnover = self._calculate_turnover(self.previous_weights, weights)
            transaction_cost = turnover * self.transaction_cost
            portfolio_return -= transaction_cost
        
        # MEJORA 39: Guardar rendimiento por factor para aprendizaje
        try:
            if hasattr(self, 'current_regime') and self.current_regime is not None:
                # Guardar rendimientos de categorías para aprendizaje futuro
                factors, factor_by_category = self.calculate_factors(date)
                if factors:
                    category_scores = self.calculate_factor_scores(factors, factor_by_category)
                    
                    for category, scores in category_scores.items():
                        # Calcular rendimiento hipotético de esta categoría
                        top_stocks = scores.sort_values(ascending=False).head(10).index
                        category_return = 0
                        
                        for ticker in top_stocks:
                            if ticker in self.stock_data and next_date in self.stock_data[ticker].index:
                                stock_return = self.stock_data[ticker].loc[next_date, 'returns']
                                if not np.isnan(stock_return):
                                    category_return += stock_return / len(top_stocks)
                        
                        # Ajustar por transacción
                        category_return -= self.transaction_cost * 0.2  # Estimación simple
                        
                        # Guardar rendimiento
                        if category not in self.factor_performance:
                            self.factor_performance[category] = []
                        
                        self.factor_performance[category].append((date, category_return))
        except Exception as e:
            logging.error(f"Error guardando rendimiento por factor: {str(e)}")
        
        return portfolio_return
    
    def _get_next_trading_date(self, date: pd.Timestamp) -> Optional[pd.Timestamp]:
        """
        Obtiene la siguiente fecha de trading disponible.
        
        Args:
            date: Fecha actual
            
        Returns:
            Siguiente fecha de trading o None si no hay datos
        """
        # Obtener todas las fechas disponibles en los datos de mercado
        market_dates = self.market_data.index
        
        # Encontrar fechas futuras
        future_dates = market_dates[market_dates > date]
        
        if len(future_dates) > 0:
            return future_dates[0]
        
        return None
    
    def run_backtest(self):
        """
        Ejecuta el backtest de la estrategia para todo el período.
        """
        print("Ejecutando backtest...")
        
        # Obtener fechas de rebalanceo
        rebalance_dates = pd.date_range(
            start=self.start_date, 
            end=self.end_date, 
            freq=self.portfolio_rebalance_frequency
        )
        
        # Filtrar a fechas disponibles en los datos
        rebalance_dates = [date for date in rebalance_dates if date in self.market_data.index]
        
        # Para almacenar resultados
        self.portfolio_weights_history = []
        portfolio_returns = []
        dates = []
        
        # Para almacenar pesos previos
        self.previous_weights = None
        
        # MEJORA 40: Agregar stop-loss dinámico
        cumulative_return = 1.0
        max_cumulative = 1.0
        in_market = True  # Indicador de si estamos invertidos o en efectivo
        
        # Ejecutar para cada fecha de rebalanceo
        for date in tqdm(rebalance_dates, desc="Procesando fechas de rebalanceo"):
            try:
                # Verificar stop-loss
                if cumulative_return < max_cumulative * 0.85:  # 15% drawdown
                    # Activar modo defensivo
                    print(f"Stop-loss activado en {date}. Drawdown: {(1 - cumulative_return/max_cumulative)*100:.2f}%")
                    in_market = False
                    
                # Verificar condición para volver al mercado
                if not in_market:
                    # Verificar si el régimen ha cambiado a favorable
                    if date in self.regimes.index and self.regimes.loc[date, 'regime'] == 0:
                        # Régimen favorable (baja volatilidad), volver al mercado
                        print(f"Volviendo al mercado en {date}. Régimen favorable detectado.")
                        in_market = True
                
                if in_market:
                    # Actualizar creencias sobre factores
                    self.update_beliefs(date)
                    
                    # Construir portafolio
                    weights = self.construct_portfolio(date)
                    
                    # Guardar pesos
                    self.portfolio_weights_history.append((date, weights))
                    
                    # Calcular retorno
                    portfolio_return = self.calculate_portfolio_return(weights, date)
                    
                    # Actualizar pesos previos
                    self.previous_weights = weights
                else:
                    # Estamos en efectivo
                    weights = pd.Series(0, index=self.previous_weights.index if self.previous_weights is not None else [])
                    self.portfolio_weights_history.append((date, weights))
                    portfolio_return = 0.0  # Considerar tasa libre de riesgo en una implementación real
                    self.previous_weights = weights
                
                # Guardar resultados
                dates.append(date)
                portfolio_returns.append(portfolio_return)
                
                # Actualizar equity
                cumulative_return *= (1 + portfolio_return)
                max_cumulative = max(max_cumulative, cumulative_return)
                
            except Exception as e:
                logging.error(f"Error en fecha {date}: {str(e)}")
                import traceback
                logging.error(traceback.format_exc())
        
        # Crear series de retornos
        self.portfolio_returns = pd.Series(portfolio_returns, index=dates)
        
        # Calcular equity curve
        self.equity_curve = (1 + self.portfolio_returns).cumprod()
        
        # Calcular métricas
        self.calculate_performance_metrics()
        
        # Guardar resultados
        self.save_results()
        
        # Visualizar resultados
        self.plot_results()
    
    def calculate_performance_metrics(self):
        """
        Calcula métricas de rendimiento de la estrategia.
        """
        returns = self.portfolio_returns
        
        # Obtener retornos del mercado para el mismo período
        market_returns = self.market_data.loc[returns.index, 'returns']
        
        # Equity curves
        equity_curve = (1 + returns).cumprod()
        market_equity = (1 + market_returns).cumprod()
        
        # Retorno anualizado
        n_years = len(returns) / 252  # Aproximadamente 252 días de trading por año
        total_return = equity_curve.iloc[-1] - 1
        annual_return = (1 + total_return) ** (1 / n_years) - 1
        
        market_total_return = market_equity.iloc[-1] - 1
        market_annual_return = (1 + market_total_return) ** (1 / n_years) - 1
        
        # Volatilidad anualizada
        volatility = returns.std() * np.sqrt(252)
        market_volatility = market_returns.std() * np.sqrt(252)
        
        # Sharpe Ratio
        risk_free_rate = 0.02  # Tasa libre de riesgo (2%)
        sharpe_ratio = (annual_return - risk_free_rate) / volatility
        market_sharpe = (market_annual_return - risk_free_rate) / market_volatility
        
        # Máximo drawdown
        cum_returns = equity_curve
        running_max = cum_returns.cummax()
        drawdown = (cum_returns / running_max) - 1
        max_drawdown = drawdown.min()
        
        market_cum_returns = market_equity
        market_running_max = market_cum_returns.cummax()
        market_drawdown = (market_cum_returns / market_running_max) - 1
        market_max_drawdown = market_drawdown.min()
        
        # Calmar Ratio (retorno anualizado / máximo drawdown absoluto)
        calmar_ratio = annual_return / abs(max_drawdown) if max_drawdown != 0 else float('inf')
        market_calmar = market_annual_return / abs(market_max_drawdown) if market_max_drawdown != 0 else float('inf')
        
        # Alpha y Beta
        # Calcular regression para beta y alpha
        X = market_returns.values.reshape(-1, 1)
        X = np.concatenate([np.ones_like(X), X], axis=1)  # Añadir columna de unos para intercepto
        y = returns.values
        
        # Eliminar NaNs
        mask = ~np.isnan(X).any(axis=1) & ~np.isnan(y)
        X_clean = X[mask]
        y_clean = y[mask]
        
        if len(X_clean) > 0 and len(y_clean) > 0:
            try:
                # Regresión lineal simple
                beta_alpha = np.linalg.lstsq(X_clean, y_clean, rcond=None)[0]
                alpha, beta = beta_alpha[0], beta_alpha[1]
                
                # Anualizar alpha
                alpha_annualized = alpha * 252
            except:
                alpha_annualized = 0.0
                beta = 1.0
        else:
            alpha_annualized = 0.0
            beta = 1.0
        
        # Guardar métricas
        self.metrics = {
            'total_return': total_return,
            'annual_return': annual_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'calmar_ratio': calmar_ratio,
            'alpha': alpha_annualized,
            'beta': beta,
            'market_return': market_total_return,
            'market_annual_return': market_annual_return,
            'market_volatility': market_volatility,
            'market_sharpe': market_sharpe,
            'market_max_drawdown': market_max_drawdown,
            'market_calmar': market_calmar
        }
    
    def save_results(self):
        """
        Guarda los resultados del backtest.
        """
        # Guardar retornos
        self.portfolio_returns.to_csv('./artifacts/results/data/portfolio_returns.csv')
        
        # Guardar equity curve
        self.equity_curve.to_csv('./artifacts/results/data/equity_curve.csv')
        
        # Guardar métricas de rendimiento
        pd.Series(self.metrics).to_csv('./artifacts/results/data/performance_metrics.csv')
        
        # Guardar pesos del portafolio a lo largo del tiempo
        weights_df = pd.DataFrame({date.strftime('%Y-%m-%d'): weights for date, weights in self.portfolio_weights_history})
        weights_df.to_csv('./artifacts/results/data/portfolio_weights_history.csv')
        
        # Guardar creencias finales sobre factores
        belief_df = pd.DataFrame([
            {
                'category': category,
                'regime': regime,
                'alpha': self.belief_distributions[category][regime]['alpha'],
                'beta': self.belief_distributions[category][regime]['beta'],
                'expected_value': self.belief_distributions[category][regime]['alpha'] / 
                                (self.belief_distributions[category][regime]['alpha'] + self.belief_distributions[category][regime]['beta'])
            }
            for category in self.factor_categories
            for regime in range(self.n_regimes)
        ])
        belief_df.to_csv('./artifacts/results/data/factor_beliefs.csv', index=False)
    
    def plot_results(self):
        """
        Genera visualizaciones de los resultados del backtest.
        """
        # Gráfico de rendimiento
        plt.figure(figsize=(15, 8))
        plt.plot(self.equity_curve, label='AMAR Strategy')
        
        # Añadir benchmark de mercado
        market_equity = (1 + self.market_data.loc[self.portfolio_returns.index, 'returns']).cumprod()
        plt.plot(market_equity, label='Market Benchmark', alpha=0.7)
        
        plt.title('Performance Comparison')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.legend()
        plt.grid(True)
        plt.savefig('./artifacts/results/figures/performance_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Gráfico de drawdown
        plt.figure(figsize=(15, 6))
        
        # Calcular drawdown
        cum_returns = self.equity_curve
        running_max = cum_returns.cummax()
        drawdown = (cum_returns / running_max) - 1
        
        market_cum_returns = market_equity
        market_running_max = market_cum_returns.cummax()
        market_drawdown = (market_cum_returns / market_running_max) - 1
        
        plt.plot(drawdown, label='AMAR Strategy', color='blue')
        plt.plot(market_drawdown, label='Market Benchmark', color='red', alpha=0.7)
        
        plt.title('Drawdown Comparison')
        plt.xlabel('Date')
        plt.ylabel('Drawdown')
        plt.legend()
        plt.grid(True)
        plt.savefig('./artifacts/results/figures/drawdown_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Histograma de retornos
        plt.figure(figsize=(12, 6))
        
        plt.hist(self.portfolio_returns, bins=50, alpha=0.5, label='AMAR Strategy')
        plt.hist(self.market_data.loc[self.portfolio_returns.index, 'returns'], bins=50, alpha=0.5, label='Market Benchmark')
        
        plt.title('Return Distribution')
        plt.xlabel('Daily Return')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True)
        plt.savefig('./artifacts/results/figures/return_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Gráfico de pesos por categoría de factores
        self.plot_factor_category_weights()
        
        # Gráfico de creencias sobre factores
        self.plot_factor_beliefs()
    
    def plot_factor_category_weights(self):
        """
        Visualiza la evolución de pesos por categoría de factores.
        """
        # Extraer fechas y pesos por categoría
        dates = [date for date, _ in self.portfolio_weights_history]
        
        # Si tenemos suficientes fechas, crear visualización
        if len(dates) > 5:
            # Para simplificar, mostraremos la evolución de pesos para categorías en últimos 20 rebalanceos
            n_samples = min(20, len(dates))
            sample_dates = dates[-n_samples:]
            
            # Crear diccionario para almacenar pesos por categoría
            category_weights = {category: [] for category in self.factor_categories}
            
            # Para cada fecha, obtener pesos por categoría (según régimen)
            for date in sample_dates:
                # Obtener régimen para esta fecha
                if self.regimes is not None and date in self.regimes.index:
                    regime = self.regimes.loc[date, 'regime']
                    
                    # Obtener pesos usando Thompson Sampling
                    weights = self.thompson_sampling(regime)
                    
                    # Guardar pesos
                    for category in self.factor_categories:
                        category_weights[category].append(weights.get(category, 0))
            
            # Crear dataframe
            weights_df = pd.DataFrame(category_weights, index=sample_dates)
            
            # Graficar
            plt.figure(figsize=(14, 7))
            weights_df.plot(kind='bar', stacked=True, ax=plt.gca(), colormap='viridis')
            
            plt.title('Factor Category Weights Evolution')
            plt.xlabel('Date')
            plt.ylabel('Weight')
            plt.legend(title='Factor Category')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig('./artifacts/results/figures/factor_category_weights.png', dpi=300, bbox_inches='tight')
            plt.close()
    
    def plot_factor_beliefs(self):
        """
        Visualiza las creencias finales sobre factores en cada régimen.
        """
        # Crear dataframe para visualización
        belief_data = []
        
        for category in self.factor_categories:
            for regime in range(self.n_regimes):
                alpha = self.belief_distributions[category][regime]['alpha']
                beta = self.belief_distributions[category][regime]['beta']
                
                # Calcular valor esperado de la distribución Beta
                expected_value = alpha / (alpha + beta)
                
                belief_data.append({
                    'Category': category,
                    'Regime': f'Regime {regime}',
                    'Expected Performance': expected_value
                })
        
        belief_df = pd.DataFrame(belief_data)
        
        # Convertir a formato wide para heatmap
        belief_matrix = belief_df.pivot(index='Category', columns='Regime', values='Expected Performance')
        
        # Crear heatmap
        plt.figure(figsize=(12, 8))
        ax = sns.heatmap(belief_matrix, annot=True, cmap='RdYlGn', fmt='.3f', linewidths=.5, cbar_kws={'label': 'Expected Performance'})
        
        plt.title('Factor Performance Beliefs by Market Regime')
        plt.tight_layout()
        plt.savefig('./artifacts/results/figures/factor_beliefs_heatmap.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Gráfico adicional: distribuciones Beta para cada combinación
        rows, cols = 3, len(self.factor_categories)
        fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
        
        for i, category in enumerate(self.factor_categories):
            for j in range(self.n_regimes):
                alpha = self.belief_distributions[category][j]['alpha']
                beta = self.belief_distributions[category][j]['beta']
                
                # Generar puntos para distribución Beta
                x = np.linspace(0, 1, 100)
                y = stats.beta.pdf(x, alpha, beta)
                
                # Graficar
                axes[j, i].plot(x, y)
                axes[j, i].set_title(f'{category} - Regime {j}')
                axes[j, i].set_xlim(0, 1)
                axes[j, i].set_ylim(0, max(y) * 1.1)
                
                # Añadir valor esperado
                expected = alpha / (alpha + beta)
                axes[j, i].axvline(expected, color='red', linestyle='--', alpha=0.6)
                axes[j, i].text(expected + 0.02, max(y) * 0.8, f'E={expected:.3f}', color='red')
        
        plt.tight_layout()
        plt.savefig('./artifacts/results/figures/factor_belief_distributions.png', dpi=300, bbox_inches='tight')
        plt.close()

        
    def run_strategy(self):
        """
        Ejecuta la estrategia completa: descarga datos, identifica regímenes,
        realiza backtest y walk-forward testing.
        """
        try:
            # 1. Descargar datos
            self.download_data()
            
            # 2. Identificar regímenes de mercado
            self.identify_regimes()
            
            # 3. Ejecutar backtest
            self.run_backtest()
                       
            # 5. Imprimir resumen de resultados
            self.print_summary()
            
        except Exception as e:
            logging.error(f"Error ejecutando estrategia: {str(e)}")
            import traceback
            logging.error(traceback.format_exc())
    
    def print_summary(self):
        """
        Imprime un resumen de los resultados de la estrategia.
        """
        if not hasattr(self, 'metrics') or not self.metrics:
            print("No hay métricas disponibles. Asegúrate de ejecutar el backtest primero.")
            return
        
        print("\n" + "="*50)
        print("RESUMEN DE RESULTADOS - ESTRATEGIA AMAR (OPTIMIZADA)")
        print("="*50)
        
        print(f"\nPeríodo de Backtest: {self.start_date} a {self.end_date}")
        print(f"Retorno Total: {self.metrics['total_return']:.2%}")
        print(f"Retorno Anualizado: {self.metrics['annual_return']:.2%}")
        print(f"Volatilidad Anualizada: {self.metrics['volatility']:.2%}")
        print(f"Sharpe Ratio: {self.metrics['sharpe_ratio']:.2f}")
        print(f"Máximo Drawdown: {self.metrics['max_drawdown']:.2%}")
        print(f"Calmar Ratio: {self.metrics['calmar_ratio']:.2f}")
        print(f"Alpha Anualizado: {self.metrics['alpha']:.2%}")
        print(f"Beta: {self.metrics['beta']:.2f}")
        
        print("\nBenchmark (Mercado):")
        print(f"Retorno Total: {self.metrics['market_return']:.2%}")
        print(f"Retorno Anualizado: {self.metrics['market_annual_return']:.2%}")
        print(f"Volatilidad Anualizada: {self.metrics['market_volatility']:.2%}")
        print(f"Sharpe Ratio: {self.metrics['market_sharpe']:.2f}")
        print(f"Máximo Drawdown: {self.metrics['market_max_drawdown']:.2%}")
        
        print("\nResultados guardados en: ./artifacts/results/")
        print("="*50)


# Función principal para ejecutar la estrategia
def run_amar_strategy(start_date='2015-01-01', end_date='2022-12-31'):
    """
    Ejecuta la estrategia AMAR mejorada con los parámetros optimizados.
    
    Args:
        start_date: Fecha de inicio del backtest
        end_date: Fecha de fin del backtest
    """
    # Crear instancia de la estrategia mejorada
    strategy = AMAR(
        start_date=start_date,
        end_date=end_date,
        lookback_years=1,
        regime_update_frequency='M',
        portfolio_rebalance_frequency='W-FRI',
        n_regimes=3,
        market_index='SPY',
        target_beta=0.,          # Estrategia neutral al mercado
        beta_range=0.1,
        max_stock_weight=0.05,    # Permitir más concentración en mejores señales
        max_sector_deviation=0.05,
        max_turnover=0.2,         # Reducir costos de transacción
        volatility_target=0.1,   # Controlar mejor el riesgo
        min_liquidity=5e6,        # Mayor umbral de liquidez
        transaction_cost=0.0015,  # Más realista (0.15%)
        random_state=42
    )
    
    # Ejecutar estrategia
    strategy.run_strategy()
    
    return strategy


if __name__ == "__main__":
    # Ejecutar estrategia mejorada
    strategy = run_amar_strategy(start_date='2024-01-01', end_date='2025-01-01')

Descargando datos para 509 activos desde 2023-01-01 hasta 2025-01-01...
YF.download() has changed argument auto_adjust default to True


Procesando activos: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503/503 [00:01<00:00, 442.13it/s]


Datos descargados y procesados para 500 activos válidos.
Identificando regímenes de mercado...
Ejecutando backtest...


Procesando fechas de rebalanceo: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [04:34<00:00,  5.37s/it]



RESUMEN DE RESULTADOS - ESTRATEGIA AMAR (OPTIMIZADA)

Período de Backtest: 2024-01-01 a 2025-01-01
Retorno Total: 1.45%
Retorno Anualizado: 7.38%
Volatilidad Anualizada: 11.43%
Sharpe Ratio: 0.47
Máximo Drawdown: -3.12%
Calmar Ratio: 2.37
Alpha Anualizado: 5.44%
Beta: 0.06

Benchmark (Mercado):
Retorno Total: 8.07%
Retorno Anualizado: 46.73%
Volatilidad Anualizada: 12.14%
Sharpe Ratio: 3.68
Máximo Drawdown: -2.67%

Resultados guardados en: ./artifacts/results/


In [17]:
print("DONE")

DONE


In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import warnings
from typing import Dict, List, Tuple, Optional

# Configuración básica
plt.style.use('seaborn-v0_8-darkgrid')
warnings.filterwarnings('ignore')

# Directorios básicos
os.makedirs('./results', exist_ok=True)

class EnhancedAMAR:
    """
    Implementación simplificada y mejorada de la estrategia AMAR
    con enfoque en rendimiento superior al mercado.
    """
    
    def __init__(self,
                 start_date: str = '2015-01-01',
                 end_date: str = None,
                 lookback_years: int = 2,
                 portfolio_rebalance_frequency: str = 'W-FRI',
                 n_regimes: int = 2,  # Simplificado a 2 regímenes
                 market_index: str = 'SPY',
                 target_beta: float = 0.7,  # Cambiado a exposición controlada al mercado
                 max_stock_weight: float = 0.07,  # Permitir mayor concentración en mejores oportunidades
                 volatility_target: float = 0.15,  # Permitir mayor volatilidad para mayor rendimiento
                 transaction_cost: float = 0.001,
                 random_state: int = 42):
        """
        Inicializa la estrategia mejorada con parámetros optimizados.
        """
        self.start_date = start_date
        self.end_date = end_date if end_date else datetime.now().strftime('%Y-%m-%d')
        self.lookback_years = lookback_years
        self.portfolio_rebalance_frequency = portfolio_rebalance_frequency
        self.n_regimes = n_regimes
        self.market_index = market_index
        self.target_beta = target_beta
        self.max_stock_weight = max_stock_weight
        self.volatility_target = volatility_target
        self.transaction_cost = transaction_cost
        self.random_state = random_state
        
        # Atributos internos simplificados
        self.data = None
        self.sp500_stocks = None
        self.market_data = None
        self.regimes = None
        self.portfolio_weights = pd.DataFrame()
        self.portfolio_returns = pd.Series(dtype='float64')
        self.current_regime = None
        
        # Simplificación de factores - Enfoque en los más efectivos
        self.factor_categories = ['Value', 'Momentum', 'Quality']
        
        # Métricas de rendimiento
        self.metrics = {}
        
        np.random.seed(random_state)
    
    def get_sp500_tickers(self) -> List[str]:
        """Obtiene tickers del S&P 500 desde Wikipedia."""
        try:
            table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
            df = table[0]
            tickers = df['Symbol'].str.replace('.', '-').tolist()
            return tickers
        except Exception as e:
            print(f"Error al obtener tickers: {str(e)}")
            # Lista reducida en caso de error
            return ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'BRK-B', 'JPM', 'JNJ', 'V', 'PG', 'UNH', 'HD']
    
    def download_data(self):
        """Descarga datos históricos con enfoque simplificado."""
        try:
            # Calcular fecha de inicio para datos históricos
            extended_start = (datetime.strptime(self.start_date, '%Y-%m-%d') - 
                             timedelta(days=int(365.25 * self.lookback_years))).strftime('%Y-%m-%d')
            
            # Obtener tickers del S&P 500
            self.sp500_stocks = self.get_sp500_tickers()
            print(f"Descargando datos para {len(self.sp500_stocks)} activos...")
            
            # Incluir índice de mercado
            tickers = self.sp500_stocks + [self.market_index]
            
            # Descargar datos
            self.data = yf.download(
                tickers, 
                start=extended_start, 
                end=self.end_date,
                group_by='ticker',
                progress=False
            )
            
            # Procesar datos del mercado
            self.market_data = pd.DataFrame({
                'close': self.data[self.market_index]['Close'],
                'returns': self.data[self.market_index]['Close'].pct_change()
            })
            
            # Procesar datos de acciones
            stocks_data = {}
            for ticker in self.sp500_stocks:
                try:
                    if ticker in self.data.columns.levels[0]:
                        stock_data = pd.DataFrame({
                            'close': self.data[ticker]['Close'],
                            'volume': self.data[ticker]['Volume'],
                        })
                        stock_data['returns'] = stock_data['close'].pct_change()
                        
                        # Solo mantener acciones con suficientes datos
                        if stock_data['close'].dropna().shape[0] > 252:
                            stocks_data[ticker] = stock_data
                except Exception as e:
                    print(f"Error procesando {ticker}: {str(e)}")
            
            self.stock_data = stocks_data
            print(f"Datos procesados para {len(stocks_data)} activos válidos.")
                
        except Exception as e:
            print(f"Error descargando datos: {str(e)}")
            raise
    
    def calculate_regime_features(self) -> pd.DataFrame:
        """Calcula características simplificadas para identificación de regímenes."""
        market = self.market_data.copy()
        
        # Características simplificadas y efectivas
        features = pd.DataFrame(index=market.index)
        
        # 1. Tendencia de precios
        market['sma50'] = market['close'].rolling(50).mean()
        market['sma200'] = market['close'].rolling(200).mean()
        
        features['trend_indicator'] = market['sma50'] / market['sma200'] - 1
        
        # 2. Volatilidad
        features['volatility'] = market['returns'].rolling(21).std() * np.sqrt(252)
        
        # 3. Momentum
        features['momentum'] = market['close'].pct_change(63)  # 3 meses
        
        # Imputar valores faltantes
        features = features.fillna(method='bfill').fillna(features.mean())
        
        return features
    
    def identify_regimes(self):
        """Identifica regímenes de mercado con enfoque simplificado."""
        print("Identificando regímenes de mercado...")
        
        # Calcular características
        features = self.calculate_regime_features()
        
        # Usar solo datos a partir de start_date
        training_data = features[features.index >= self.start_date].copy()
        
        if training_data.shape[0] > 30:
            # Normalizar características
            scaler = StandardScaler()
            normalized_data = scaler.fit_transform(training_data)
            
            # Ajustar GMM simplificado
            gmm = GaussianMixture(
                n_components=self.n_regimes,
                covariance_type='full',
                random_state=self.random_state,
                n_init=10
            )
            
            gmm.fit(normalized_data)
            labels = gmm.predict(normalized_data)
            
            # Ordenar regímenes por volatilidad (0: baja, 1: alta)
            regime_volatility = {}
            for i in range(self.n_regimes):
                mask = (labels == i)
                if mask.sum() > 0:
                    regime_volatility[i] = training_data.loc[mask, 'volatility'].mean()
            
            sorted_regimes = sorted(regime_volatility.items(), key=lambda x: x[1])
            regime_map = {old: new for new, (old, _) in enumerate(sorted_regimes)}
            
            # Reasignar etiquetas
            new_labels = np.array([regime_map[label] for label in labels])
            
            # Crear DataFrame de regímenes
            self.regimes = pd.DataFrame(index=training_data.index, columns=['regime'])
            self.regimes['regime'] = new_labels
            
            # Simple plot para verificar regímenes
            self._plot_simple_regimes()
        else:
            print("Datos insuficientes para identificar regímenes.")
    
    def _plot_simple_regimes(self):
        """Visualización simple de regímenes."""
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Graficar precio de mercado
        ax.plot(self.market_data.loc[self.regimes.index, 'close'], 'k-', label='Mercado')
        
        # Colorear regímenes
        colors = ['green', 'red']  # 0: baja volatilidad, 1: alta volatilidad
        labels = ['Baja Volatilidad', 'Alta Volatilidad']
        
        # Agrupar regímenes consecutivos
        for regime in range(self.n_regimes):
            regime_periods = self.regimes[self.regimes['regime'] == regime].index
            if len(regime_periods) > 0:
                ax.fill_between(regime_periods, 0, 1, 
                               transform=ax.get_xaxis_transform(),
                               alpha=0.3, color=colors[regime], label=labels[regime])
        
        ax.set_title('Regímenes de Mercado')
        ax.legend()
        plt.tight_layout()
        plt.savefig('./results/market_regimes.png', dpi=150)
        plt.close()
    
    def calculate_factors(self, date: pd.Timestamp) -> Dict[str, pd.Series]:
        """Calcula factores simplificados pero efectivos."""
        # Períodos de lookback
        lookback_long = date - pd.Timedelta(days=365)
        lookback_short = date - pd.Timedelta(days=63)
        
        # Filtrar acciones con datos suficientes
        valid_stocks = [ticker for ticker, data in self.stock_data.items() 
                       if data.loc[:date].shape[0] > 252]
        
        factors = {}
        
        # 1. VALOR (Price to máximo histórico como proxy)
        pb_proxy = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_long:date]
            if data.shape[0] > 0:
                current_price = data['close'].iloc[-1]
                historical_high = data['close'].max()
                if historical_high > 0:
                    pb_proxy[ticker] = current_price / historical_high
        factors['Value'] = -pd.Series(pb_proxy)  # Invertido para que valores más altos sean mejores
        
        # 2. MOMENTUM (3, 6, 12 meses con ponderación)
        weighted_momentum = {}
        periods = [(63, 0.4), (126, 0.3), (252, 0.3)]  # períodos y pesos
        
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[:date]
            if data.shape[0] > 252:
                momentum_sum = 0.0
                weight_sum = 0.0
                
                for days, weight in periods:
                    if data.shape[0] >= days:
                        ret = data['close'].iloc[-1] / data['close'].iloc[-min(days, len(data))] - 1
                        momentum_sum += ret * weight
                        weight_sum += weight
                
                if weight_sum > 0:
                    weighted_momentum[ticker] = momentum_sum / weight_sum
                    
        factors['Momentum'] = pd.Series(weighted_momentum)
        
        # 3. CALIDAD (Estabilidad de retornos - menor drawdown)
        quality = {}
        for ticker in valid_stocks:
            data = self.stock_data[ticker].loc[lookback_long:date]
            if data.shape[0] > 126:
                # Calidad basada en resistencia a caídas
                cum_returns = (1 + data['returns']).cumprod()
                if len(cum_returns) > 0:
                    max_dd = (cum_returns / cum_returns.cummax() - 1).min()
                    quality[ticker] = -max_dd  # Invertir para que valores más altos sean mejores
                    
        factors['Quality'] = pd.Series(quality)
        
        # Normalizar y controlar outliers para todos los factores
        normalized_factors = {}
        for factor_name, factor_values in factors.items():
            if len(factor_values) > 0:
                # Winsorización para controlar outliers
                lower_bound = factor_values.quantile(0.02)
                upper_bound = factor_values.quantile(0.98)
                winsorized = factor_values.clip(lower=lower_bound, upper=upper_bound)
                
                # Z-score normalization
                mean = winsorized.mean()
                std = winsorized.std()
                if std > 0:
                    normalized = (winsorized - mean) / std
                    normalized_factors[factor_name] = normalized
        
        return normalized_factors
    
    def get_factor_weights(self, date: pd.Timestamp) -> Dict[str, float]:
        """Determina pesos dinámicos para factores según régimen de mercado."""
        # Si no hay regímenes identificados, usar pesos iguales
        if self.regimes is None or date not in self.regimes.index:
            return {factor: 1.0/len(self.factor_categories) for factor in self.factor_categories}
        
        # Obtener régimen actual
        regime = self.regimes.loc[date, 'regime']
        
        # Asignar pesos según régimen - optimizado basado en evidencia empírica
        if regime == 0:  # Baja volatilidad / alcista
            return {
                'Value': 0.2,      # Valor menos importante en mercados alcistas
                'Momentum': 0.55,  # Momentum muy efectivo en tendencias claras
                'Quality': 0.25    # Calidad siempre importante
            }
        else:  # Alta volatilidad / bajista
            return {
                'Value': 0.35,     # Valor más importante en mercados volátiles
                'Momentum': 0.15,  # Momentum menos fiable en alta volatilidad
                'Quality': 0.5     # Calidad crucial en mercados estresados
            }
    
    def construct_portfolio(self, date: pd.Timestamp) -> pd.Series:
        """Construye portafolio con enfoque simplificado pero efectivo."""
        # Calcular factores
        factors = self.calculate_factors(date)
        
        if not factors:
            return pd.Series(dtype='float64')
        
        # Obtener pesos de factores según régimen
        factor_weights = self.get_factor_weights(date)
        
        # Combinar factores con pesos
        combined_score = pd.Series(0, index=next(iter(factors.values())).index)
        
        for factor, weight in factor_weights.items():
            if factor in factors:
                combined_score += factors[factor] * weight
        
        # Seleccionar mejores acciones (top 15%)
        num_stocks = max(20, int(0.15 * len(combined_score)))
        top_stocks = combined_score.nlargest(num_stocks).index
        
        # Asignar pesos basados en ranking
        weights = pd.Series(0, index=combined_score.index)
        
        # Pesos proporcionales al score
        for i, stock in enumerate(top_stocks):
            weights[stock] = combined_score[stock]
        
        # Normalizar y aplicar peso máximo
        if weights.sum() > 0:
            weights = weights / weights.sum()
            weights = weights.clip(upper=self.max_stock_weight)
            weights = weights / weights.sum()
        
        # Aplicar controles de riesgo
        weights = self._apply_risk_controls(weights, date)
        
        return weights
    
    def _apply_risk_controls(self, weights: pd.Series, date: pd.Timestamp) -> pd.Series:
        """Aplica controles de riesgo simplificados y efectivos."""
        # Si no hay pesos, retornar serie vacía
        if weights.sum() == 0:
            return weights
        
        # 1. Control de Beta
        try:
            # Calcular betas individuales
            betas = {}
            lookback_start = date - pd.Timedelta(days=252)
            
            for ticker in weights.index[weights > 0]:
                if ticker in self.stock_data:
                    stock_data = self.stock_data[ticker].loc[lookback_start:date]
                    market_slice = self.market_data.loc[lookback_start:date]
                    
                    common_dates = stock_data.index.intersection(market_slice.index)
                    if len(common_dates) > 60:
                        stock_returns = stock_data.loc[common_dates, 'returns']
                        market_returns = market_slice.loc[common_dates, 'returns']
                        
                        # Eliminar NaN
                        valid_mask = ~(np.isnan(stock_returns) | np.isnan(market_returns))
                        stock_returns = stock_returns[valid_mask]
                        market_returns = market_returns[valid_mask]
                        
                        if len(stock_returns) > 60:
                            # Calcular beta simple
                            cov = np.cov(stock_returns, market_returns)[0, 1]
                            var = np.var(market_returns)
                            if var > 0:
                                betas[ticker] = cov / var
            
            # Ajustar portafolio si tenemos suficientes betas
            if len(betas) > 10:
                beta_series = pd.Series(betas)
                portfolio_beta = (weights.loc[beta_series.index] * beta_series).sum()
                
                # Solo ajustar si estamos muy lejos del objetivo
                if abs(portfolio_beta - self.target_beta) > 0.2:
                    # Ordenar acciones por beta
                    sorted_betas = beta_series.sort_values()
                    
                    if portfolio_beta > self.target_beta + 0.2:
                        # Reducir exposición a acciones de alto beta
                        high_beta = sorted_betas.tail(int(len(sorted_betas) * 0.3))
                        for ticker in high_beta.index:
                            weights[ticker] *= 0.7
                    elif portfolio_beta < self.target_beta - 0.2:
                        # Aumentar exposición a acciones de alto beta
                        high_beta = sorted_betas.tail(int(len(sorted_betas) * 0.3))
                        for ticker in high_beta.index:
                            weights[ticker] *= 1.3
                    
                    # Renormalizar
                    weights = weights / weights.sum()
        except Exception as e:
            print(f"Error en control de beta: {str(e)}")
        
        # 2. Control de Volatilidad (simplificado)
        try:
            # Estimar volatilidad del portafolio
            stock_vol = {}
            for ticker in weights.index[weights > 0]:
                if ticker in self.stock_data:
                    returns = self.stock_data[ticker].loc[date-pd.Timedelta(days=63):date, 'returns']
                    if len(returns) > 30:
                        stock_vol[ticker] = returns.std() * np.sqrt(252)
            
            if stock_vol:
                # Estimación simple de volatilidad de portafolio
                vol_series = pd.Series(stock_vol)
                avg_vol = (weights.loc[vol_series.index] * vol_series).sum()
                
                # Escalar si volatilidad estimada es demasiado alta
                if avg_vol > self.volatility_target * 1.5:
                    scale = self.volatility_target / avg_vol
                    weights *= scale
        except Exception as e:
            print(f"Error en control de volatilidad: {str(e)}")
        
        return weights
    
    def run_backtest(self):
        """Ejecuta backtest con lógica simplificada."""
        print("Ejecutando backtest...")
        
        # Obtener fechas de rebalanceo
        rebalance_dates = pd.date_range(
            start=self.start_date, 
            end=self.end_date, 
            freq=self.portfolio_rebalance_frequency
        )
        
        # Filtrar a fechas disponibles en datos
        rebalance_dates = [date for date in rebalance_dates if date in self.market_data.index]
        
        # Almacenar resultados
        portfolio_weights_history = []
        portfolio_returns = []
        dates = []
        
        previous_weights = None
        
        # Ejecutar para cada fecha de rebalanceo
        for date in rebalance_dates:
            try:
                # Construir portafolio
                weights = self.construct_portfolio(date)
                
                # Guardar pesos
                portfolio_weights_history.append((date, weights))
                
                # Calcular retorno
                next_date = self._get_next_trading_date(date)
                if next_date:
                    portfolio_return = 0.0
                    
                    for ticker, weight in weights.items():
                        if ticker in self.stock_data and next_date in self.stock_data[ticker].index:
                            stock_return = self.stock_data[ticker].loc[next_date, 'returns']
                            if not np.isnan(stock_return):
                                portfolio_return += weight * stock_return
                    
                    # Aplicar costo de transacción si tenemos pesos previos
                    if previous_weights is not None:
                        turnover = self._calculate_turnover(previous_weights, weights)
                        transaction_cost = turnover * self.transaction_cost
                        portfolio_return -= transaction_cost
                    
                    # Guardar resultados
                    dates.append(date)
                    portfolio_returns.append(portfolio_return)
                
                # Actualizar pesos previos
                previous_weights = weights
                
            except Exception as e:
                print(f"Error en fecha {date}: {str(e)}")
        
        # Crear series de retornos
        self.portfolio_returns = pd.Series(portfolio_returns, index=dates)
        
        # Calcular equity curve
        self.equity_curve = (1 + self.portfolio_returns).cumprod()
        
        # Calcular métricas
        self.calculate_performance_metrics()
        
        # Visualizar resultados
        self.plot_results()
    
    def _calculate_turnover(self, old_weights: pd.Series, new_weights: pd.Series) -> float:
        """Calcula turnover entre conjuntos de pesos."""
        all_stocks = old_weights.index.union(new_weights.index)
        
        old_unified = pd.Series(0, index=all_stocks)
        new_unified = pd.Series(0, index=all_stocks)
        
        old_unified.loc[old_weights.index] = old_weights
        new_unified.loc[new_weights.index] = new_weights
        
        turnover = np.sum(np.abs(old_unified - new_unified)) / 2
        
        return turnover
    
    def _get_next_trading_date(self, date: pd.Timestamp) -> Optional[pd.Timestamp]:
        """Obtiene siguiente fecha de trading disponible."""
        market_dates = self.market_data.index
        future_dates = market_dates[market_dates > date]
        
        if len(future_dates) > 0:
            return future_dates[0]
        
        return None
    
    def calculate_performance_metrics(self):
        """Calcula métricas de rendimiento simplificadas."""
        returns = self.portfolio_returns
        
        # Obtener retornos del mercado para el mismo período
        market_returns = self.market_data.loc[returns.index, 'returns']
        
        # Equity curves
        equity_curve = (1 + returns).cumprod()
        market_equity = (1 + market_returns).cumprod()
        
        # Retorno anualizado
        n_years = len(returns) / 252
        total_return = equity_curve.iloc[-1] - 1
        annual_return = (1 + total_return) ** (1 / n_years) - 1
        
        market_total_return = market_equity.iloc[-1] - 1
        market_annual_return = (1 + market_total_return) ** (1 / n_years) - 1
        
        # Volatilidad anualizada
        volatility = returns.std() * np.sqrt(252)
        market_volatility = market_returns.std() * np.sqrt(252)
        
        # Sharpe Ratio (asumiendo tasa libre de riesgo 0 para simplificar)
        sharpe_ratio = annual_return / volatility
        market_sharpe = market_annual_return / market_volatility
        
        # Máximo drawdown
        running_max = equity_curve.cummax()
        drawdown = (equity_curve / running_max) - 1
        max_drawdown = drawdown.min()
        
        market_running_max = market_equity.cummax()
        market_drawdown = (market_equity / market_running_max) - 1
        market_max_drawdown = market_drawdown.min()
        
        # Calcular beta
        cov = np.cov(returns, market_returns)[0, 1]
        var = np.var(market_returns)
        if var > 0:
            beta = cov / var
        else:
            beta = 1.0
        
        # Alpha simple
        alpha = annual_return - (beta * market_annual_return)
        
        # Guardar métricas
        self.metrics = {
            'total_return': total_return,
            'annual_return': annual_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'beta': beta,
            'alpha': alpha,
            'market_return': market_total_return,
            'market_annual': market_annual_return
        }
    
    def plot_results(self):
        """Genera visualizaciones simplificadas de resultados."""
        # Único gráfico de rendimiento con métricas
        plt.figure(figsize=(12, 8))
        
        # Graficar equity curves
        plt.plot(self.equity_curve, label='Estrategia Mejorada', linewidth=2)
        
        # Graficar benchmark
        market_equity = (1 + self.market_data.loc[self.portfolio_returns.index, 'returns']).cumprod()
        plt.plot(market_equity, label='S&P 500', alpha=0.7, linewidth=1.5)
        
        # Añadir texto con métricas clave
        metrics_text = (
            f"Estrategia:\n"
            f"Retorno Anual: {self.metrics['annual_return']:.2%}\n"
            f"Volatilidad: {self.metrics['volatility']:.2%}\n"
            f"Sharpe: {self.metrics['sharpe_ratio']:.2f}\n"
            f"Max DD: {self.metrics['max_drawdown']:.2%}\n\n"
            f"S&P 500:\n"
            f"Retorno Anual: {self.metrics['market_annual']:.2%}\n"
            f"Alpha: {self.metrics['alpha']:.2%}\n"
            f"Beta: {self.metrics['beta']:.2f}"
        )
        
        # Posición del texto: 80% del eje x, 20% del eje y
        plt.annotate(metrics_text, xy=(0.02, 0.02), xycoords='axes fraction', 
                    bbox=dict(boxstyle="round,pad=0.5", fc="white", alpha=0.8))
        
        plt.title('Rendimiento de la Estrategia Mejorada vs S&P 500')
        plt.xlabel('Fecha')
        plt.ylabel('Crecimiento de $1')
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('./results/performance.png', dpi=200)
        plt.close()
    
    def run_strategy(self):
        """Ejecuta la estrategia completa."""
        try:
            # 1. Descargar datos
            self.download_data()
            
            # 2. Identificar regímenes
            self.identify_regimes()
            
            # 3. Ejecutar backtest
            self.run_backtest()
            
            # 4. Imprimir resumen
            self.print_summary()
            
        except Exception as e:
            print(f"Error ejecutando estrategia: {str(e)}")
    
    def print_summary(self):
        """Imprime resumen de resultados."""
        if not self.metrics:
            print("No hay métricas disponibles.")
            return
        
        print("\n" + "="*50)
        print("RESUMEN DE RESULTADOS - ESTRATEGIA MEJORADA")
        print("="*50)
        
        print(f"\nPeríodo: {self.start_date} a {self.end_date}")
        print(f"Retorno Total: {self.metrics['total_return']:.2%}")
        print(f"Retorno Anualizado: {self.metrics['annual_return']:.2%}")
        print(f"Volatilidad: {self.metrics['volatility']:.2%}")
        print(f"Sharpe Ratio: {self.metrics['sharpe_ratio']:.2f}")
        print(f"Máximo Drawdown: {self.metrics['max_drawdown']:.2%}")
        print(f"Beta: {self.metrics['beta']:.2f}")
        print(f"Alpha: {self.metrics['alpha']:.2%}")
        
        print("\nS&P 500:")
        print(f"Retorno Total: {self.metrics['market_return']:.2%}")
        print(f"Retorno Anualizado: {self.metrics['market_annual']:.2%}")
        
        print("\nResultados guardados en: ./results/")
        print("="*50)

def run_enhanced_strategy(start_date='2020-01-01', end_date='2025-01-01'):
    """Ejecuta la estrategia mejorada con parámetros optimizados."""
    strategy = EnhancedAMAR(
        start_date=start_date,
        end_date=end_date,
        target_beta=0.7,          # Exposición controlada al mercado
        volatility_target=0.15,   # Mayor tolerancia a volatilidad para mayor rendimiento
        max_stock_weight=0.07,    # Mayor concentración en mejores oportunidades
        transaction_cost=0.001    # Costos realistas
    )
    
    strategy.run_strategy()
    return strategy

if __name__ == "__main__":
    strategy = run_enhanced_strategy()

Descargando datos para 503 activos...
Datos procesados para 500 activos válidos.
Identificando regímenes de mercado...
Ejecutando backtest...

RESUMEN DE RESULTADOS - ESTRATEGIA MEJORADA

Período: 2020-01-01 a 2025-01-01
Retorno Total: 4.12%
Retorno Anualizado: 4.12%
Volatilidad: 9.90%
Sharpe Ratio: 0.42
Máximo Drawdown: -11.20%
Beta: -0.01
Alpha: 4.38%

S&P 500:
Retorno Total: 19.51%
Retorno Anualizado: 19.51%

Resultados guardados en: ./results/


In [7]:
print("DONE")

DONE
